Ref: https://zhuanlan.zhihu.com/p/67192636

https://tf.wiki/zh_hant/advanced/static.html

https://zhuanlan.zhihu.com/p/403383500

https://github.com/lyhue1991/eat_tensorflow2_in_30_days/blob/master/4-3%2CAutoGraph%E7%9A%84%E4%BD%BF%E7%94%A8%E8%A7%84%E8%8C%83.md

https://www.tensorflow.org/api_docs/python/tf/autograph


有三種計算圖的構建方式：靜態計算圖，動態計算圖，以及Autograph.

- 1.在TensorFlow1.0時代，採用的是靜態計算圖，需要先使用TensorFlow的各種算子創建計算圖，然後再開啟一個會話Session，顯式執行計算圖。

- 2.而在TensorFlow2.0時代，採用的是動態計算圖，即每使用一個算子後，該算子會被動態加入到隱含的默認計算圖中立即執行得到結果，而無需開啟Session。

- 3.如果需要在TensorFlow2.0中使用靜態圖，可以使用@tf.function裝飾器將普通Python函數轉換成對應的TensorFlow計算圖構建代碼。運行該函數就相當於在TensorFlow1.0中用Session執行代碼。使用tf.function構建靜態圖的方式叫做 Autograph.

Autograph編碼規範解析

一，Autograph編碼規範總結

- 1.被@tf.function修飾的函數應盡可能使用TensorFlow中的函數而不是Python中的其他函數。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

- 2.避免在@tf.function修飾的函數內部定義tf.Variable.

- 3.被@tf.function修飾的函數不可修改該函數外部的Python列表或字典等數據結構變量。

In [1]:
#1.被@tf.function修飾的函數應盡量使用TensorFlow中的函數而不是Python中的其他函數。
import numpy as np
import tensorflow as tf

@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
    
@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [2]:
#np_random每次執行都是一樣的結果。
np_random()
np_random()

array([[-1.73540585, -0.54000727, -0.53008478],
       [-0.88393372, -1.12192669,  0.30468015],
       [ 1.08796699,  1.48895093, -1.86873079]])
array([[-1.73540585, -0.54000727, -0.53008478],
       [-0.88393372, -1.12192669,  0.30468015],
       [ 1.08796699,  1.48895093, -1.86873079]])


In [3]:
#tf_random每次執行都會有重新生成隨機數。
tf_random()
tf_random()

[[0.537724912 1.03548372 0.650293589]
 [1.09334135 -0.463545 1.09124362]
 [-0.157314405 0.295352876 -0.494494677]]
[[1.42983747 0.790028512 -0.354884803]
 [-1.13240695 -1.26856434 -1.22019017]
 [-0.307600409 0.546139777 -0.895069361]]


In [4]:
#2.避免在@tf.function修飾的函數內部定義tf.Variable.

#為何不用在函數內部定義的說明: https://blog.csdn.net/qq_40913465/article/details/104604979
x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [5]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

#執行將報錯
#inner_var()
#inner_var()

In [6]:
#3.被@tf.function修飾的函數不可修改該函數外部的Python列表或字典等數據結構變量。
tensor_list = []

#@tf.function #加上這一行切換成Autograph結果將不符合預期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [7]:
tensor_list = []

@tf.function #加上這一行切換成Autograph結果將不符合預期！！！
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
